In [ ]:
import json
def load_jsonl(file_path, sort_by_id = True):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            json_obj = json.loads(line.strip())
            # break
            data.append(json_obj)
    # print('list has', len(data), 'lines')
    if sort_by_id:
        return sorted(data, key=lambda x: x['id'])
    return data

In [ ]:
retriever_keys = ['wiki_id_match', 'wiki_par_id_match']
reader_keys = ['substring_match', 'exact_match']
retriever_file = '/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/evaluations/bm25/nq-dev-kilt.jsonl'
reader_file = '/data/tir/projects/tir6/general/afreens/dbqa/reader_results/reader_output_evaluated_with_baselines.jsonl'
retriever_text_file = '/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/predictions/bm25/nq-dev-kilt.jsonl'

retriever_list = load_jsonl(retriever_file)
reader_list = load_jsonl(reader_file)
retriever_text_list = load_jsonl(retriever_text_file)


In [ ]:
import json
import numpy as np

def get_pipeline_zeno_results(retriever_list, retriever_text_list, reader_list, k):
    compiled_results = []
    # retriever_results = []
    # reader_results = []

    for l_id, (retriever_out, retriever_texts, reader_out) in enumerate(zip(retriever_list, retriever_text_list, reader_list)):
        doc_results = {}
        doc_results['id'] = retriever_out['id']
        doc_results['question'] = retriever_texts['input']
        doc_results['output'] = {'context': []}
        
        # retriever_results_per_doc = []
        # reader_results_per_doc = []
        if (retriever_out['id']!= reader_out['id']):
                print(l_id, 'diff id')
        any_wiki_id_match = False
        any_wiki_par_id_match = False
        for prov_id, (rt_out, rt_text, rd_out) in enumerate(zip(retriever_out['doc-level results'], retriever_texts['output'][0]['provenance'], reader_out['output']['provenance'])):
            if (rt_out['wiki_par_id']!= rd_out['docid']):
                print(prov_id, 'diff wiki id')
            if (rt_text['docid'] != rd_out['docid']):
                 print(prov_id, 'diff wiki id')
            if prov_id == k:
                break
            if (prov_id == 0):
                 doc_results['output']['answer'] = rd_out['answer']
                 doc_results['output']['answer evaluation'] = rd_out['answer_evaluation']
            if rt_out['wiki_id_match']:
                 any_wiki_id_match = True
            if rt_out['wiki_par_id_match']:
                 any_wiki_par_id_match = True
            context_info = {'wiki_id': rt_out['wiki_id'], 'wiki_par_id': rt_out['wiki_par_id']}
            context_info['wiki_id_match'] = rt_out['wiki_id_match']
            context_info['wiki_par_id_match'] = rt_out['wiki_par_id_match']
            context_info['text'] = rt_text['text']
            context_info['score'] = rt_text['score']
            doc_results['output']['context'].append(context_info)
        
        doc_results['output']['summary context evaluation'] = {'wiki_id_match': any_wiki_id_match, \
                                                       'wiki_par_id_match': any_wiki_par_id_match}
        reordered_keys = ['answer', 'answer evaluation', 'context', 'summary context evaluation']
        doc_results['output'] = {k: doc_results['output'][k] for k in reordered_keys}

        compiled_results.append(doc_results)

            
    return compiled_results

In [ ]:
compiled_results = get_pipeline_zeno_results(retriever_list, retriever_text_list, reader_list, 1)

In [ ]:
with open('/data/tir/projects/tir6/general/afreens/dbqa/compiled_zeno_results/top-1_bm25_llama_nq_dev_kilt_results.json', 'w') as file:
    json.dump(compiled_results, file, indent=4)